In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def process_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    edges = cv2.Canny(blur, 50, 150)

    def region_of_interest(frame):
        height = frame.shape[0]
        polygon = np.array([[(100, height), (900, height), (550, 250)]])
        mask = np.zeros_like(frame)
        cv2.fillPoly(frame, polygon, 255)
        return cv2.bitwise_and(frame, mask)
    
    roi_image = region_of_interest(edges)

    lines = cv2.HoughLinesP(
        roi_image, 
        rho=2, 
        theta=np.pi/180, 
        threshold=100, 
        lines=np.array([]), 
        minLineLength=40, 
        maxLineGap=5
    )

    def draw_lines(img, lines):
        line_image = np.zeros_like(img)
        if lines is not None:
            for line in lines:
                x1, x2, y1, y2 = line[0]
                cv2.line(img, (x1, y1), (x2, y2), (255, 0, 0), 60)
        return cv2.addWeighted(img, 0.8, line_image, 1, 1)
    
    return draw_lines(frame, lines)

In [14]:
input_video_path = r"C:\Projects\Lane_Detection\80400-572395752_tiny.mp4"
output_video_path = 'output_video.avi'

cap = cv2.VideoCapture(input_video_path)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_video_path, fourcc, 20.0, 
                      (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
                       int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

In [15]:
while cap.isOpened():
    r, frame = cap.read()
    if r:
        processed_frame = process_frame(frame)
        out.write(processed_frame)
        cv2.imshow('Lane Detection', processed_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
out.release()
cv2.destroyAllWindows()